# Votre SDK en 2 minutes ⏱!

#### N.D.L.R.: Pour la démo…
<!-- .slide: data-background="#2E2E2E" data-state="nologo-slide" style="text-align: center;" -->
… j'ai choisi *CodeGen* pour la diversité des languages de sortie qu'il propose.

Mais Azure/Autorest ou OpenAPI Generator font tout aussi bien le travail.

### Prérequis

On créé un dossier pour notre démo avec un sous dossier pour les "`in`puts" et un autre pour les les "`out`puts".

In [ ]:
!mkdir -p codegen/in/ codegen/out/
!ls codegen/

## Swagger-codegen / docker

On récupère l'image docker de `swagger-codegen`:

In [ ]:
!docker pull swaggerapi/swagger-codegen-cli-v3

Quels langages sont supportés en sortie:

In [ ]:
!docker run --rm swaggerapi/swagger-codegen-cli-v3 langs | sed 's/,/\n    /g'

## La météo en 2 minutes

![By Openweather Ltd. - https://openweathermap.org/, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=86908006](./images/OpenWeather-Logo.jpg)

La doc de l'API est disponible sur le site officiel, ou sinon sur: [app.swaggerhub.com](https://app.swaggerhub.com/apis/IdRatherBeWriting/open-weather_map_api/2.5.2)


Ici on va:

1. récupérer la doc de l'API,
2. la formater un peu (`| python -m json.tool`)
3. la stocker
4. afficher quelques lignes

In [ ]:
%%bash
# On récupère le descripteur de l'API
curl -sk https://app.swaggerhub.com/apiproxy/registry/IdRatherBeWriting/open-weather_map_api/2.5.2 \
  | python -m json.tool \
  > codegen/in/open-weather_map_api.json
head -n 25 codegen/in/open-weather_map_api.json

In [ ]:
%%bash
# On prépare un fichier de configuration pour notre futur module
echo '{
  "packageName":"owm_client",
  "projectName":"owm-client",
  "packageVersion":"2.5.2"
}' > codegen/in/config_owm.json

In [ ]:
%%bash
# On génère notre SDK
docker run --rm -v ${PWD}/codegen:/local \
  swaggerapi/swagger-codegen-cli-v3 generate \
    -i /local/in/open-weather_map_api.json \
    -o /local/out/python-owm \
    -c /local/in/config_owm.json \
    -l python

Le résultat c'est un nouveau module python !

In [ ]:
%%bash
# Un rapide server web pour accéder à la documentation
(cd codegen/out/python-owm ; python -m http.server 9998)

[Le lien vers la documentation](http://vm:9998)

In [ ]:
%%bash
# On installe le nouveau module
pip install codegen/out/python-owm

In [ ]:
import owm_client
from owm_client.rest import ApiException
from utils import *
import os
logger = logging.getLogger("DEMO_OPENWEATHERMAP")

# Configure API
configuration = owm_client.Configuration()

# credentials
configuration.api_key['appid'] = os.environ.get('OWM_API_TOKEN')

# Create a client
api_instance = owm_client.CurrentWeatherDataApi(owm_client.ApiClient(configuration))
city = 'Bordeaux'
lang = 'fr'
mode = 'json'
units = 'metric'

try:
    # Call current weather data for one location
    api_response = api_instance.current_weather_data(q=city, units=units, lang=lang, mode=mode)
    logger.info("Data:\n" +
        json.dumps(api_response.to_dict(), indent=2, default=str)
    )
except ApiException as e:
    print("Exception when calling CurrentWeatherDataApi->current_weather_data: %s\n" % e)

## Chuck Norris

![chucknorrisio_logo.png](./images/chucknorrisio_logo.png)<!-- .element style="max-width:50%;" -->
<!-- .slide: class="centered" -->

In [ ]:
%%bash

# On récupère le descripteur de l'API
curl -s https://api.chucknorris.io/documentation \
    | python -m json.tool > codegen/in/chucknorris.json

# On prépare un fichier de configuration pour notre futur module
echo '{
  "packageName":"chucknorris_client",
  "projectName":"chucknorris-client",
  "packageVersion":"1.0.0"
}' > codegen/in/config_chucknorris_client.json

# On génère notre SDK
docker run --rm -v ${PWD}/codegen:/local \
  swaggerapi/swagger-codegen-cli-v3 generate \
    -i /local/in/chucknorris.json \
    -o /local/out/python-chucknorris \
    -c /local/in/config_chucknorris_client.json \
    -l python

# On installe le nouveau module
pip install codegen/out/python-chucknorris/

In [ ]:
import chucknorris_client
from chucknorris_client.rest import ApiException

from utils import *
logger = logging.getLogger("DEMO_CHUCKNORRIS")

# Configure API
logger.debug("Create an API client")
client = chucknorris_client.ApiClient(chucknorris_client.Configuration())

logger.debug("Target the Joke Controller")
api_instance = chucknorris_client.JokeControllerApi(client)

try:
    logger.debug("Get a random joke:")
    api_response = api_instance.get_random_joke_value_using_get()
    logger.info(api_response)
except ApiException as e:
    logger.error(
        "Exception when calling JokeControllerApi->get_random_joke_value_using_get: %s\n" % e
    )

### En Go ?

In [ ]:
%%bash
docker run --rm -v ${PWD}/codegen:/local \
  swaggerapi/swagger-codegen-cli-v3 generate \
    -i /local/in/chucknorris.json \
    -o /local/out/gochucknorris \
    -DpackageName=gochucknorris \
    -l go

In [ ]:
%%bash
echo 'package main

import (
    "log"
    cn "out/gochucknorris"
    "golang.org/x/net/context"
)

func main() {
    log.Print("New API client with empty configuration")
    client := cn.NewAPIClient(cn.NewConfiguration())
    log.Print("Get random joke")
    joke, r, err := client.JokeControllerApi.GetRandomJokeValueUsingGET(context.Background(), nil)
    // Test error
    if err != nil {
        log.Print("Error:", err)
    }
    // Test HTTP Response code
    if r.StatusCode == 200 {
        log.Print(joke.Value)
    }
}' > codegen/chuck.go

In [ ]:
!go run codegen/chuck.go

![vmware.png](./images/vmware.png)<!-- .element style="max-width:35%;" -->
## vCenter

La même chose en plus rapide avec l'API REST de VMware vCenter, un produit un peu plus complexe:

🗣️
> Lancer l'exécution avant les explications ! 

In [ ]:
%%bash
# On récupère le descripteur de l'API
curl -sk https://$VMWARE_VCENTER_HOSTNAME/apiexplorer/json/vcenter.json  \
  > codegen/in/vcenter.json

# On prépare un fichier de configuration pour notre futur module
echo '{
  "packageName":"vc_client",
  "projectName":"vc-client",
  "packageVersion":"6.7.0"
}' > codegen/in/config_vc_client.json

# On génère notre SDK
docker run --rm -v ${PWD}/codegen:/local \
  swaggerapi/swagger-codegen-cli-v3 generate \
    -i /local/in/vcenter.json \
    -o /local/out/python-vc \
    -c /local/in/config_vc_client.json \
    -l python

# On installe le nouveau module
pip install codegen/out/python-vc

In [ ]:
import vc_client
from vc_client.rest import ApiException
from utils import *
import os
logger = logging.getLogger("DEMO_VCENTER")

# Configure API
configuration = vc_client.Configuration()
configuration.verify_ssl = False
configuration.host = f"https://{os.environ.get('VMWARE_VCENTER_HOSTNAME')}/rest"

# credentials
_username = os.environ.get('VMWARE_VCENTER_USERNAME')
_password = os.environ.get('VMWARE_VCENTER_PASSWORD')
auth_str = basic_auth_str(_username, _password)

# Create a client
client = vc_client.ApiClient(configuration)

In [ ]:
# Get a new session
try:
    logger.debug("Starting a new session")
    s = client.call_api(
        '/com/vmware/cis/session',
        "POST",
        header_params={
            "Authorization": auth_str,
        })
except ApiException as e:
    print("Exception when creating session: %s\n" % e)
    exit(-1)
logger.info("New session is created")

# Set the cookie according to the previous request result
logger.debug("Setting new session authorization token in cookies")
client.cookie = s[2].get('Set-Cookie')
logger.info(f"Client cookies updated: {client.cookie}")

In [ ]:
# List VM
logger.debug("Listing VMs...")
instance = vc_client.VM_Api(client)
for vm in instance.list().value:
    logger.info(
        f"{vm.name}: {vm.power_state} / vCPU: {vm.cpu_count} / Mem: {vm.memory_size_mi_b} Mb"
    )
    keep_last = vm

In [ ]:
# Get more details for last VM
logger.debug("Getting details about last VM...")
vm_detailled = instance.get(keep_last.vm).value
logger.info("Data:\n" +
            json.dumps(vm_detailled.to_dict(), indent=2, default=str))

In [ ]:
# Get its network
logger.debug("Getting a specifc detail about a VM:")
nic = vm_detailled.nics[0].value
logger.info(f"{keep_last.name} MAC address: {nic.mac_address}")

![vmware.png](./images/vmware.png)<!-- .element style="max-width:35%;" -->
## vCloud Director

Second test avec vCloud Director.

In [ ]:
%%bash
# On récupère le descripteur de la "cloud API"
curl -sk https://$VMWARE_VCD_HOSTNAME/api-explorer/tenant/orgdemo02/cloudapi.json \
  > codegen/in/cloudapi.json

# On prépare un fichier de configuration pour notre futur module
echo '{
  "packageName":"vcd_client",
  "projectName":"vcd-client",
  "packageVersion":"9.7.1"
}' > codegen/in/config_vcd_client.json

# On génère notre SDK
docker run --rm -v ${PWD}/codegen:/local \
  swaggerapi/swagger-codegen-cli-v3 generate \
    -i /local/in/cloudapi.json \
    -o /local/out/python-vcd \
    -c /local/in/config_vcd_client.json \
    -l python

# On installe le nouveau module
pip install codegen/out/python-vcd

In [ ]:
import vcd_client
from vcd_client.rest import ApiException
from utils import *
import os
logger = logging.getLogger("DEMO_VCD")

# Configure API
configuration = vcd_client.Configuration()
configuration.verify_ssl = False
configuration.host = f"https://{os.environ.get('VMWARE_VCD_HOSTNAME')}/cloudapi"

# credentials
_username = os.environ.get('VMWARE_VCD_USERNAME')
_password = os.environ.get('VMWARE_VCD_PASSWORD')
auth_str = basic_auth_str(_username, _password)

# Create a client
client = vcd_client.ApiClient(configuration)

In [ ]:
# Get a new session
try:
    logger.debug("Starting a new session")
    s = vcd_client.SessionsApi(client)
    s_headers = s.login_with_http_info(authorization=auth_str)[2]
except ApiException as e:
    print("Exception when creating session: %s\n" % e)
    exit(-1)
logger.info("New session is created")

# Update client with access token
logger.debug("Setting new session authorization token in headers")
configuration.api_key_prefix['Authorization'] = 'Bearer'
configuration.api_key['Authorization'] = s_headers.get(
    "X-VMWARE-VCLOUD-ACCESS-TOKEN"
)
client = vcd_client.ApiClient(configuration)
logger.info(f"Client credentials updated to use access token: {s_headers.get('X-VMWARE-VCLOUD-ACCESS-TOKEN')}")

In [ ]:
# List rights of the current user
logger.debug("Getting rights of the current user")
rapi = vcd_client.RightsApi(client)
page, page_size = 1, 25
try:
    for right in rapi.query_rights(page, page_size).values:
        logger.info(" - ".join([right.name, right.id, right.right_type]))
except ApiException as e:
    logger.error("Exception when calling RightsApi->query_rights: %s\n" % e)